## 96 well raw data growth experiment analysis


In [ ]:
# import block
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from tqdm import tqdm
import re
import os
from matplotlib_venn import venn3
import math
import sys
import multiprocessing as mp
from time import time, sleep
import openpyxl
import statistics
from scipy.stats import gompertz
from datetime import timedelta, datetime
import numpy as np
from scipy import stats
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from tkinter import filedialog
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc, dash_table
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
'''# google drive api

from __future__ import print_function

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError'''




plt.rcParams['svg.fonttype'] = 'none'

Raw data file parsing and virtual plate layout creation

In [ ]:
''' We have a couple of different instrument so first I need to understand the file layout coming from each one
From a quick checkup it looks like the files are streaming out from the same software - Gen5 so int heory they should look the same unless the export promts are different.'''

# read file to count the lines to skip and lines to read. Also try to identify the column rang automatically

# ---------- get files from a shared folder on google drive - using drive desktop -----------

# read into a preprocess df
# store file names in a variable for easy use downstream
root = tk.Tk()
root.withdraw()
# this is for dev and debug purposes. IRL the files will be grabbed from a google drive folder
ipath='G:/Shared drives/Cultivarium/Culture/Data analysis results/'
showinfo('Plate results', 'Please choose the experiments results file')

filepath = filedialog.askopenfilename(initialdir=ipath)

# get the file name and get rid of the extension
filename=filepath.replace(ipath,'')

filename=filename.split('.')
filename=filename[0]

# create the results and inside create the graphics folder
respath=ipath+'/results_'+filename
if not os.path.isdir(respath):
    os.mkdir(respath)
graphicspath=respath+'/graphics'
if not os.path.isdir(graphicspath):
    os.mkdir(graphicspath)


# read in layout file and parse it
# note: add filename to all saved files
showinfo('Plate layout', 'Please choose the plate layout file')
layoutfile = filedialog.askopenfilename(initialdir=ipath)
# deal with multiple blanks and condition titles
samples={}
col2sample={}
blanks={}
conditions={}
def get_wells(wgstr, sampledict):
    wstr=wgstr.split('\t')
    sample=wstr[0]
    triplicates=wstr[1].split(',')
    triplicates=[s.strip() for s in triplicates]
    sampledict[sample]=triplicates
    for t in triplicates:
        col2sample[t]=sample
    
bflag=False
sflag=False
with open(layoutfile) as lin:
    for line in lin:
        line=line.replace('"','')
        if 'blanks' in line:
            bflag=True
            continue
        if '///' not in line and bflag:
            if '\\\\\\' in line:
                bflag=False
                continue
            get_wells(line,blanks)
        if 'samples' in line:
            sflag=True
            bflag=False
            continue
        if '///' not in line and sflag:
            if '\\\\\\' in line:
                sflag=False
                continue
            get_wells(line,samples)
        if 'conditions' in line:
            sflag= False
            bflag=False
            continue
        if '///' not in line and not (bflag or sflag):
            line=line.split('\t')
            if '\\\\\\' in line[0]:
                continue
            conditions[line[0].strip()]=line[1].strip()




ppdf=pd.read_excel(filepath)
cols=ppdf.columns
#print(cols[0])
# get the plate id, this will be the connector identifier between the plate and plate sample annotation files
plateidobj=ppdf[ppdf[cols[0]]=='Plate Number']
plateid=plateidobj.iloc[0,1]
print('Plate ID is: '+plateid)
# Get the line number of the 315,415 string
# start line number
sln=ppdf.index[ppdf[cols[0]]==315415]
#print(sln[0])
# get the end line number
eln=ppdf.index[ppdf[cols[0]]=='Results']
#print(eln[0])
nr=(eln[0]-sln[0])-4
#print(nr)
# starting from the synergy H1 format
expdf=pd.read_excel(filepath, skiprows=sln[0]+3, nrows=nr, usecols="B:CU")
#platedf.to_csv('test_input.csv')

# add a minutes column for easy plotting and calculations
minuteslist=[]
for t in expdf['Time']:
    t0=expdf.loc[0]['Time']
    hrs=t.hour
    minutes=t.minute
    seconds=t.second
    # calculate minutes
    tminutes=(hrs*60)+minutes+seconds/60
    minuteslist.append(tminutes)
    
# add the column to grapdf
minuteseries=pd.Series(minuteslist, name='total_minutes')
expdf=pd.concat([expdf,minuteseries],axis=1)

expdf


# Plate raw data overview

In [ ]:
# general plots

# plot all the wells in subplots to look like the plate
def plotPlate(plate, title):
    
    # plots multiple subplots of  a given dataframe
    i=0
    # use plate columns to get the number of columns and rows for the subplots
    nrows=set()
    ncols=set()
    for col in plate.columns:
        if any(x in col for x in['Time','T° 315,415','total_minutes']):
            continue  
        w=re.match("([A-Z])([0-9]+)",col)
        nrows.add(w.group(1))
        ncols.add(w.group(2))
    fig=plt.figure(figsize=(2.5*len(ncols),2.5*len(nrows)))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    # Add the main title
    fig.suptitle(title, fontsize=30)
    for col in plate.columns:
        # skip the time and temperature column
        if any(x in col for x in['Time','T° 315,415','total_minutes']):
            continue    
        # go down a row every 12 wells
        i+=1
        '''if i%12==0:
            i=1'''
        ax=fig.add_subplot(len(nrows),len(ncols),i)
        ax.plot(plate['total_minutes'], plate[col])
        if col in col2sample:
            plt.title(col2sample[col],fontweight="bold")
        else:
            plt.title(col)
        plt.xlabel('Time (minutes)')
        plt.ylabel('OD600')
    plt.tight_layout()
    plt.savefig(graphicspath+'/'+title+'_'+filename +'.svg')
    plt.show()

plotPlate(expdf,'Raw data')       

# temperature distribution across time
cnames=expdf.columns

expdf.plot(x=cnames[0], y=cnames[1], figsize=(10,5), color='grey',style=".")
plt.title('Temperature change with time')
plt.ylabel('Temperature, Celsius')
plt.savefig(graphicspath+'/tempvstime_'+filename+'.svg')
plt.show()

plt.figure()
expdf[cnames[1]].plot.kde(figsize=(10,5), title='Temperature distribution in the experiment')
plt.xlabel('Temperature, Celsius')
plt.savefig(graphicspath+'/temp_dist_'+filename+'.svg')
plt.show()



# Blank subtraction
After reading the plate, identify samples by loading a sample file

In [ ]:

# this will have the output table
workdf = pd.DataFrame()
cleandf=pd.DataFrame()
wd={}
cdict={}
nc_err_names=[]
nc_avg_names=set()
ncdict = {}
blank2samples={}
# ---------------analyze plate---------------------------

# plot all the wells in both semilog (y) and regular plots

for i, tp in tqdm(expdf.iterrows()):
    ncdict[tp['total_minutes']] = {}
    wd[tp['total_minutes']]={}
    cdict[tp['total_minutes']]={}
    # calculate the average  and standard deviation per sample
    
    # blanks
    for b in blanks:
        curcols=[]
        vals=[]
        for nc in blanks[b]:
            vals.append(tp[nc])
        nc_avg=statistics.mean(vals)
        nc_stdev=statistics.stdev(vals)
        nca_name = b+'_avg'
        ncs_name = b+'_stdev'
        ncdict[tp['total_minutes']][nca_name] = nc_avg
        ncdict[tp['total_minutes']][ncs_name] = nc_stdev
        nc_err_names.append(ncs_name)
        nc_avg_names.add(nca_name)
        
        # find the columns matching the blank, the blanks are named NC_sample
        bsample=b.split('_')
        cursample=bsample[1].strip()
        # get the column list for the matching sample
        curcols=samples[cursample]
        blank2samples[b]=curcols
        
        # create a clean version for column selecting
        for ccol in curcols:
            #ccol_name = ccol+'_mb'
            cdict[tp['total_minutes']][ccol] =tp[ccol]-nc_avg
        # add the blank columns
        curcols=curcols+blanks[b]
        
        
        
        # subtract the blank average from each column value and keep in a new table
        
        for ccol in curcols:
            
            
            #ccol_name = ccol+'_mb'
            wd[tp['total_minutes']][ccol] =tp[ccol]-nc_avg

ncdf = pd.DataFrame.from_dict(ncdict, orient='index')
ncdf.index=ncdf.index.set_names(['total_minutes'])
ncdf.reset_index(inplace=True)

# get the new plate after subtracting the blanks - blank wells should show zero values
workdf = pd.DataFrame.from_dict(wd, orient='index')
workdf.index=workdf.index.set_names(['total_minutes'])
workdf.reset_index(inplace=True)

cleandf = pd.DataFrame.from_dict(cdict, orient='index')
cleandf.index=cleandf.index.set_names(['total_minutes'])
cleandf.reset_index(inplace=True)
# plot the blanks for a closer look


plt.figure()
plt.rcParams.update({'font.size': 16}) 
ncdf.plot(x='total_minutes',  y=list(nc_avg_names),yerr=ncdf[nc_err_names].T.values,
             figsize=(20,10),
             ylabel='OD600',
             title='Blanks '+plateid,
             #logy=True,
             fontsize=14,
             #color=colors
             )
plt.legend()
plt.tight_layout()
plt.savefig(graphicspath+'/blanks'+filename+'.svg')
plt.show()


# plot the samples after blank subtraction
plotPlate(workdf,'Raw data after blank')  


#plot all the curves on the same axis set
plt.figure()
plt.rcParams.update({'font.size': 16}) 
workdf.plot(x='total_minutes',
             figsize=(20,10),
             ylabel='OD600',
             title='all curves '+plateid,
             #logy=True,
             fontsize=14,
             #color=colors
             )
plt.legend()
plt.tight_layout()
plt.savefig(graphicspath+'/all_wells_after_blank'+filename+'.svg')
plt.show()
workdf


# Add log2 values 

In [ ]:
# add log2 values, this time no need to work with the blank columns
nsamples={}
log2tdf=pd.DataFrame()
log2tdf=pd.concat([log2tdf,workdf['total_minutes']],axis=1)
for sample in list(samples):
    if 'NC' in sample or sample in nsamples:
        continue
    # add the new sample name to the samples dictionary for easy plotting
    nsample_name=sample+'_log2'
    nsamples[nsample_name]=set()
    for well in samples[sample]:
        nwell_name=well+'00'
        nsamples[nsample_name].add(nwell_name)
        wclog2=np.log2(workdf[well].values)
        col2sample[nwell_name]=nsample_name
        # add to the table
        wclog2series=pd.Series(wclog2,name=nwell_name)
        log2tdf=pd.concat([log2tdf,wclog2series],axis=1)
        log2tdf=pd.concat([log2tdf,workdf[well]],axis=1)
# update the samples dictionary
samples.update(nsamples)

#plot the new 'plate'
plotPlate(log2tdf,'Raw and log2 after blank')  

'''# save the dataframe to a file with the sample titles
sample_names_io=['total_minutes']
for col in log2tdf.columns:
    if 'total_minutes' in col:
        continue
    sample_names_io.append(col2sample[col])
outdf=log2tdf
# change column names in output table
outdf.columns=sample_names_io
outdf.to_csv(respath+'/'+'sample_data_wlog2_'+filename+'.csv')
outdf'''
log2tdf


# Get metrics for all the sample wells curves

In [ ]:
# calculate maxOD and time to reach maxOD for the wells in log2tdf
# the final table will look like 
# well | maxOD | ttr_maxOD | mid_log OD | ttr_mid_logOD | growth_rate | doubeling_time

def getMetrics(curve1,curve2, col):
    # returns the mid-log OD and time to reach mid log
    # uses rolling regression and R values to capture different segments of the curve
    
    # rolling regression - get the R values for a regression line in every coordinate
    # curve is a list of curve values over time
    #work with the smallest window - 2 coordinates
    window=2
    slopes = []
    intercepts = []
    Rs=[]
    # get the values for the window
    for c in range(len(curve1)-window):
        ogx=curve1.loc[c:c+window]
        ogy=curve2.loc[c:c+window]
        y=np.array(ogy.values.tolist())
        x=np.array(ogx.values.tolist())
        y=y.reshape(-1,1)
        x=x.reshape(-1,1)

        lr=LinearRegression()
        lr.fit(x,y)
        R_coef=lr.score(x,y)
        if R_coef<0.95:
            '''print(col+'\t'+str(c)+' minutes')
            print(R_coef)'''
            exp=c
            break
            
        Rs.append(R_coef)
        intercepts.append(lr.intercept_)
        slopes.append(lr.coef_[0])
    ogxlist=list(curve1[0:c])
    ogylist=list(curve2[0:c])
    tslope, intercept, r_value, p_value, std_err = stats.linregress(ogxlist,ogylist)    
    '''# plot vs. Time
    
    plt.figure(figsize=(10,10))
    ogxlist=list(curve1[0:c])
    ogylist=list(curve2[0:c])
    tslope, intercept, r_value, p_value, std_err = stats.linregress(ogxlist,ogylist)
    g=sns.regplot(x=ogxlist,y=ogylist,line_kws={'label':"y={0:.5f}x+{1:.1f}".format(tslope,intercept)})
    #plt.yscale('log')
    #plt.xscale('log')
    plt.legend()
    plt.show()'''
    
    # get the metrics
    exp_phase_idx=c
    midlog_idx=int(c/2)
    growth_rate=tslope
    dbl_time=1/growth_rate
    
    
    return exp_phase_idx,midlog_idx,growth_rate,dbl_time
fpopdf=pd.DataFrame()
for col in log2tdf.columns:
    if '00' in col or 'total_minutes' in col:
        continue
    # get the values for the maxOD, midlog, growth rate and doubling time
    ep_idx,mdlg_idx,gr,dbt=getMetrics(log2tdf['total_minutes'],log2tdf[col],col)
    # use the indices to get the values from the series
    # for max OD I need to use the original columns
    ogcol=col.replace('00','')
    ogcol_list=list(log2tdf[ogcol].values)
    max_od=max(log2tdf[ogcol].values)
    # get the values for ttr_maxOD,and ttr to midlog
    ttr_max_idx=log2tdf.index[log2tdf[ogcol]==max_od].values
    ttr_max_idx=ttr_max_idx[0]
    ttr_max_od=log2tdf.loc[ttr_max_idx]['total_minutes']
    
    midlog_od=ogcol_list[mdlg_idx]
    ttr_midlog=log2tdf.loc[mdlg_idx]['total_minutes']
    
    
    # update the table
    values_to_add = { 'sample_name':col2sample[ogcol],'well': ogcol,'plate_id':plateid,'maxOD600':max_od,
                     'Growth_rate':gr,'Doubling_time':dbt,'Time_to_reach_maxOD600_(minutes)':ttr_max_od,
                     'Mid_log_OD600':midlog_od,'Time_to_reach_mid_log_(minutes)':ttr_midlog }
    row_to_add = pd.Series(values_to_add, name=ogcol)
    new_df=pd.DataFrame([row_to_add], columns=row_to_add._stat_axis)
    fpopdf = pd.concat([fpopdf,new_df],ignore_index=True)
    
fpopdf
    
    

# Plot the plate in a per metric look for column selection

In [ ]:
# plot a single metric look of the samples
def plotPlateSM(plate,metric):
    
    # plot a heatmap of the sample/well vs a metric
    # use plate columns to get the number of columns and rows for the subplots
    nrows=set()
    ncols=set()
    
    tempdict={}
    tempdf=pd.DataFrame()
    vals=[]
    letters=[]
    numbers=set()
    for col in plate['well']:
        # skip the time and temperature column
        '''if any(x in col for x in['sample_name']):
            continue'''   
        w=re.match("([A-Z])([0-9]+)",col)
        letter=w.group(1)
        nrows.add(letter)
        num=w.group(2)
        numbers.add(num)
        met_idx=plate.index[plate['well']==col].values
        met_idx=met_idx[0]
        val=plate.loc[met_idx][metric]
        if letter not in tempdict:
            tempdict[letter]=[]
        tempdict[letter].append(val)
        
    numbers=list(sorted(numbers))    
    # build the df por plotting
    tempdf=pd.DataFrame.from_dict(tempdict, orient='index',columns=numbers)
    # plot
    plt.figure(figsize=(2.5*len(numbers),2.5*len(nrows)))
    ax=sns.heatmap(tempdf, cmap='Spectral')
    ax.tick_params(axis='y', rotation=0)
    plt.title(metric)
    plt.ylabel('Plate rows')
    plt.xlabel('Plate columns')
    plt.tight_layout()
    plt.savefig(graphicspath+'/'+metric+'_'+filename +'.svg')
    plt.show()          
# plot all the metrics in the fpopdf table
for met in fpopdf.columns:
    # skip the metadata
    if any(x in met for x in ['sample_name','well','plate_id']):
        continue
    plotPlateSM(fpopdf,met)

# Select columns to remove from the table
The code will drop two columns for each choice - one for the od values and one for the log2 values

In [ ]:
# select columns to remove from a dataframe - using a clean version of the data without the log2
# creating a widget for viewing the dataframe

app = JupyterDash(__name__)
# Load Data
df = cleandf
# Build App
app.layout = html.Div([
            dash_table.DataTable(
                id='table',
                columns=[
                {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
                ],
                data=df.to_dict('records'),
                editable=True,
                filter_action="native",
                sort_action="native",
                sort_mode="multi",
                column_selectable="multi",
                row_selectable="multi",
                row_deletable=True,
                selected_columns=[],
                selected_rows=[],
                page_action="native",
                page_current= 0,
                page_size= 10,
            ),
            html.Div(id='datatable-interactivity-container'),
            html.Button(id="save-button",n_clicks=0,children="Save"),
            html.Div(id="output-1",children="Press button to save changes")
])

@app.callback(
        Output("output-1","children"),
        [Input("save-button","n_clicks")],
        [State("table","data")]
        )

def selected_data_to_csv(nclicks,table1): 
    if nclicks == 0:
        raise PreventUpdate
    else:
        pd.DataFrame(table1).to_csv(respath+'/interim_df.csv',index=False)
        return "Data processed, continue running notebook"

# Run app and display result inline in the notebook
app.run_server(mode='inline', debug=True)



# Continue analyzing with consolidated, cleaned data

Plotting the growth curves

In [ ]:
# get the updated dataframe
graphdf=pd.read_csv(respath+'/interim_df.csv')
sample2col={}
# update the parameter table after deletion of columns
# now the columns are stored in the well column in the parameter table
# get the updated column list
ucol_list=list(graphdf.columns)
for cc in list(fpopdf['well']):
    if cc not in ucol_list:
        fpopdf.drop(fpopdf[fpopdf['well']==cc].index,inplace=True)
        
# create an update sample2col mapping
for cc in list(fpopdf['well']):
    dfline=fpopdf[fpopdf['well']==cc]
    sample=dfline['sample_name'].values
    sample=sample[0]
    if sample not in sample2col:
        sample2col[sample]=set()
    sample2col[sample].add(cc)
    
fpopdf
        

# Consolidate curve and parameter data
Calculate the average and standard deviation of sample using the wells that are left.

In [ ]:
# start with the curves

wd={}
s_err_names=set()
s_avg_names=set()
for i, tp in tqdm(graphdf.iterrows()):
    wd[tp['total_minutes']]={}
    # calculate the average  and standard deviation per sample
    
    # samples
    for s in sample2col:
        vals=[]
        for well in sample2col[s]:
            vals.append(tp[well])
        s_avg=statistics.mean(vals)
        if len(sample2col[s])<2:
            s_stdev=0
        else:
            s_stdev=statistics.stdev(vals)
        ns_name = s+'_avg'
        nst_name = s+'_stdev'
        wd[tp['total_minutes']][ns_name] = s_avg
        wd[tp['total_minutes']][nst_name] = s_stdev
        s_err_names.add(nst_name)
        s_avg_names.add(ns_name)
stl=list(s_err_names)
sl=list(s_avg_names)
ngraphdf = pd.DataFrame.from_dict(wd, orient='index')
ngraphdf.index=ngraphdf.index.set_names(['total_minutes'])
ngraphdf.reset_index(inplace=True)


#plot
plt.figure()
plt.rcParams.update({'font.size': 16}) 
ngraphdf.plot(x='total_minutes', y=sl,yerr=ngraphdf[stl].T.values, 
             figsize=(20,10),
             ylabel='OD600 (log)',
             title='Growth curves for '+plateid,
             logy=True,
             fontsize=14,
             )

plt.tight_layout()
plt.savefig(graphicspath+'/growth_curves_'+filename+'.svg')
plt.show() 

# plot regular axis
#plot
plt.figure()
plt.rcParams.update({'font.size': 16}) 
ngraphdf.plot(x='total_minutes', y=sl,yerr=ngraphdf[stl].T.values, 
             figsize=(20,10),
             ylabel='OD600',
             title='Growth curves for '+plateid,
             #logy=True,
             fontsize=14,
             )

plt.tight_layout()
plt.savefig(graphicspath+'/growth_curves_regaxis_'+filename+'.svg')
plt.show() 

# now do the parameters

wpd={}
errdict={}

for col in fpopdf.columns:
    
    if any(x in col for x in ['sample_name', 'well','plate_id']):
        continue
    # now we have the parameters columns and we can go over the samples and get the values
    # calculate the average  and standard deviation per sample
    
    # samples
    for s in sample2col:
        if s not in wpd:
            wpd[s]={}
        vals=[]
        for well in sample2col[s]:
            # get the index of the well
            idx=fpopdf.index[fpopdf['well']==well].values
            idx=idx[0]
            val=fpopdf.loc[idx][col]
            vals.append(val)
        s_avg=statistics.mean(vals)
        if len(sample2col[s])<2:
            s_stdev=0
        else:
            s_stdev=statistics.stdev(vals)
        ns_name = col+'_avg'
        nst_name = col+'_stdev'
        wpd[s][ns_name] = s_avg
        wpd[s][nst_name] = s_stdev
        errdict[ns_name]=nst_name

npdf = pd.DataFrame.from_dict(wpd, orient='index')
npdf.index=npdf.index.set_names(['sample'])
npdf.reset_index(inplace=True)
mcolors=['dodgerblue','goldenrod','forestgreen','crimson','mediumorchid','orange','deepskyblue','orangered','navy','lightseagreen','deeppink']

lbdict={'maxOD600_avg': 'OD',
        'Growth_rate_avg':'1/Minutes',
        'Doubling_time_avg':'Minutes',
        'Time_to_reach_maxOD600_(minutes)_avg':'Minutes',
        'Mid_log_OD600_avg':'OD',
        'Time_to_reach_mid_log_(minutes)_avg':'Minutes'   
}

# parameter plots
for l,prm in enumerate(errdict):
    
    plt.figure(figsize=(10,10))
    plt.rcParams.update({'font.size': 16}) 
    #sns.scatterplot(x='sample',y=prm,data=npdf, color=mcolors[l])
    plt.errorbar(x=npdf['sample'], y=npdf[prm], yerr=npdf[errdict[prm]],color=mcolors[l])
    plt.title(prm)
    plt.xlabel('Sample')
    plt.ylabel(lbdict[prm])
    plt.tight_layout()
    plt.savefig(graphicspath+'/params_cons'+filename+'.svg')
    plt.show()
npdf

In [ ]:
# gompertz fit
    # c represents the last measurment, x is time
def gompertz_model(x,a,b,c):
    
    t1=c*np.exp(-b*np.exp(-x/a))
    return t1

def Gompertz(t, theta1, theta2, theta3):
    '''
    theta1: The asymptote.
    theta2: The displacement along the x-axis.
    theta3: The growth rate.
    '''
    f = theta1 * np.exp(-theta2 * np.exp(-theta3 * t))
    return f

def slope(x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    return m

colors=['red','blue','grey','blue']
plt.figure()
plt.rcParams.update({'font.size': 16}) 
graphdf.plot(x='total_minutes', y=sample_names,yerr=graphdf[err_names].T.values, 
             figsize=(20,10),
             ylabel='OD600',
             title='Growth curves for '+plateid,
             logy=True,
             fontsize=14,
             color=colors)
colors=['red','blue','grey','blue']
#growth rate
gr=[0.003,0.001,0.001]
# plot in the mid-log phases
for i,sample in enumerate(sample_names):
    if 'NC' in sample:
        continue
    # gompertz c value, the ods
    lst=graphdf[sample].values
    # gompertz time
    mlst=graphdf['total_minutes'].values
    # add gompertz
    #glst=gompertz_model(mlst,0,max(lst),lst)
    # calculates the parameters for the gompertz dist
    glst, pcov_confirmed = curve_fit(f = gompertz_model, 
                                           xdata = mlst, 
                                           ydata = lst, 
                                           p0 = [max(lst), lst[0], gr[i]], 
                                           maxfev = 8000)
    

    '''print(glst)
    print('\n\n')
    print(pcov_confirmed)'''
    plt.plot(mlst,Gompertz(mlst,*glst), label=sample+' Gompertz fit')
    
    # add gls to the dataframe
    glstseries=pd.Series(glst,name=sample+' Gompertz fit')
    graphdf=pd.concat([graphdf,glstseries],axis=1)

plt.legend()
plt.tight_layout()
plt.savefig('./graphics/growth_curves.svg')
plt.show()


plt.figure()
plt.rcParams.update({'font.size': 16}) 
graphdf.plot(x='total_minutes', y=sample_names,yerr=graphdf[err_names].T.values, 
             figsize=(20,10),
             ylabel='OD600',
             title='Growth curves for '+plateid,
             #logy=True,
             fontsize=14,
             color=colors)
colors=['red','blue','grey','blue']
#growth rate
gr=[0.003,0.001,0.001]
# plot in the mid-log phases
for i,sample in enumerate(sample_names):
    if 'NC' in sample:
        continue
    # gompertz c value, the ods
    lst=graphdf[sample].values
    # gompertz time
    mlst=graphdf['total_minutes'].values
    # add gompertz
    #glst=gompertz_model(mlst,0,max(lst),lst)
    # calculates the parameters for the gompertz dist
    glst, pcov_confirmed = curve_fit(f = gompertz_model, 
                                           xdata = mlst, 
                                           ydata = lst, 
                                           p0 = [max(lst), lst[0], gr[i]], 
                                           maxfev = 8000)
    

    '''print(glst)
    print('\n\n')
    print(pcov_confirmed)'''
    plt.plot(mlst,Gompertz(mlst,*glst), label=sample+' Gompertz fit')
    
    # add gls to the dataframe
    glstseries=pd.Series(glst,name=sample+' Gompertz fit')
    graphdf=pd.concat([graphdf,glstseries],axis=1)

plt.legend()
plt.tight_layout()
plt.savefig('./graphics/growth_curves.svg')
plt.show()




Next step would be to create heatmap views of the growth rate and od - we have only 3 samples 

In [ ]:
# max od heatmap
plotdf=npdf[['maxOD600_avg','Mid_log_OD600_avg']]
plt.figure(figsize=(10,10))
ax=sns.heatmap(plotdf, cmap='Greens')
ax.tick_params(axis='y', rotation=0)
plt.title('OD overview')
plt.tight_layout()
plt.savefig(graphicspath+'/od_heatmap'+filename+'.svg')
plt.show()

# time to reach
plotdf=npdf[['Time_to_reach_maxOD600_(minutes)_avg','Time_to_reach_mid_log_(minutes)_avg']]
plt.figure(figsize=(10,10))
ax=sns.heatmap(plotdf, cmap='Blues')
ax.tick_params(axis='y', rotation=0)
ax.tick_params(axis='x',rotation=45)
plt.title('time to reach overview')
plt.tight_layout()
plt.savefig(graphicspath+'ttr_heatmap'+filename+'.svg')
plt.show()